<a href="https://colab.research.google.com/github/louissiller/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [28]:
# Replace with your actual class repo URL
#CLASS_REPO_URL = "https://github.com/YOUR-ORG-OR-USERNAME/class-repo.git"
#CLASS_REPO_DIR = "/content/class-repo"

# If already cloned, skip re-downloading
#import os
#if not os.path.exists(CLASS_REPO_DIR):
#    !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
#else:
#    %cd {CLASS_REPO_DIR}
#    !git pull

# Move into repo folder
#%cd {CLASS_REPO_DIR}

#print("✅ Class repository ready at", CLASS_REPO_DIR)

In [29]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [30]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [31]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "mgmt467-71800"   # <-- EDIT ME
# assert PROJECT_ID != "mgmt467-71800", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: mgmt467-71800


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [32]:
DATASET = "lab1_foundation"
TABLE   = "superstore_clean"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [33]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


Using Colab cache for faster access to the 'global-superstore' dataset.
Path to dataset files: /kaggle/input/global-superstore
Files:
- /kaggle/input/global-superstore/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [34]:
sql_str = f"""
SELECT DISTINCT sub_category
FROM `{PROJECT_ID}.{DATASET}.{TABLE}`
WHERE region = 'West'
ORDER BY sub_category
"""

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line:
SELECT DISTINCT `Sub-Category`
FROM `mgmt467-71800.lab1_foundation.superstore`
WHERE Region = 'West'
ORDER BY `Sub-Category`*  
Summarize what you learned here:

> **Notes:**
Basically, we're telling BigQuery to SELECT DISTINCT Sub-Category, which means "show us all the different types of sub-categories without repeating any." Then, FROM \mgmt467-71800.lab1_foundation.superstore`` just tells it where to look – in our Superstore sales table in BigQuery. The WHERE Region = 'West' part is super important; it filters the results so we only see the sub-categories from the 'West' region. Finally, ORDER BY \Sub-Category`` just sorts the results alphabetically so it's easy to read. In summary, we learned how to use SELECT DISTINCT to get unique values, FROM to specify our data source (and how to reference BigQuery tables correctly with project, dataset, and table names, and backticks for column names with hyphens!), and WHERE to filter our data based on a condition. This is like the foundation for pulling specific info out of big datasets!

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [35]:
sql_str3b = """
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `mgmt467-71800.lab1_foundation.superstore_clean`
GROUP BY ship_mode
ORDER BY order_count DESC
"""

df_howmany = bq.query(sql_str3b).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [36]:
sql_str = f"""
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `{PROJECT_ID}.{DATASET}.{TABLE}`
GROUP BY customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

**Challenge 1:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is '{PROJECT_ID}.{DATASET}.{TABLE}'. The category identifier is "Technology".
# GOAL: Calculate the average discount for products in the category "Technology" sold in the region "East".
```

In [37]:
sql_str = f"""
SELECT AVG(Discount) AS avg_discount
FROM `{PROJECT_ID}.{DATASET}.{TABLE}`
WHERE Category = 'Technology' AND Region = 'East'
"""

df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

# Display the result in the requested format
avg_discount_value = df_ch1['avg_discount'].iloc[0]
print(f"The average discount for category 'Technology' in region 'East' is {avg_discount_value}")

,avg_discount
0,0.143364


The average discount for category 'Technology' in region 'East' is 0.14336448598130852


**Challenge 2:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is '{PROJECT_ID}.{DATASET}.{TABLE}'. The identifier is "Segment".
# GOAL: Count, how many 'customer_id' have been served for segment 'Consumer' and how many have been served for segment 'corporate' and for segment 'home_office'. Count each segment individually.

In [38]:
sql_str = f"""
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `{PROJECT_ID}.{DATASET}.{TABLE}`
GROUP BY Segment
ORDER BY unique_customers DESC
"""

df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,409
1,Corporate,236
2,Home Office,148


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
I learned how to connect Colab to BigQuery and pull data using SQL. The first query showed us all the unique product sub-categories in the West region.
- **Investigate:** What alternate query or filter changed the story?  
Going through the different questions ("What", "How many", "Who is best") really showed how changing the SELECT, WHERE, GROUP BY, and ORDER BY clauses can totally change the results. For instance, switching from just listing things to counting them with COUNT(*) and GROUP BY gave us a completely different view of the data, like seeing which ship mode is used the most.
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
We saw errors with column names and the table name. The key to validating was running the queries and looking really closely at the error messages. BigQuery's error messages were actually super helpful in pointing out exactly where the syntax was wrong or if a column/table wasn't found. Comparing the generated SQL to the table schema (which we saw a preview of) and the error messages helped figure out what needed fixing.
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?
The list of sub-categories in the West region could go to the West regional manager – they could use it to see if they're stocking the right products or if there are any gaps. The "How many orders by ship mode" data could be useful for the logistics or operations team to understand shipping patterns and maybe optimize costs. And the "Top 5 most profitable customers" is definitely something the sales or marketing team would want. They could use that info to create targeted campaigns or loyalty programs for those high-value customers.

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.